In [31]:
import pandas as pd
import random
import datetime
from tqdm import tqdm
import requests as r
from bson import ObjectId

# 0. Entendiendo el problema

Variables de entrada:
- TPU
- Cantidad requerida
- Tiempo de produccion requerido

Salida:
- Personal requerido


Por lo que se tiene una relacion lineal multivariada:

`y = B0 + B1([Cantidad material]) + B2([Tiempo de fabricacion del material]) + E`

- B0: Cantidad base de personal
- B1: Un producto de el factor de incremento de personal por incremento de cantidad de material y el tpu
- B2: Tiempo requerido para fabricar

In [32]:
_mat = [
    'Discos de freno',
    'Pastillas de freno',
    'Calipers de freno',
    'Tambores de freno',
    'Zapatas de freno',
    'Cilindro maestro de freno',
    'Líneas de freno',
    'Sensor de freno ABS',
    'Servo o booster de freno',
    'Mangueras de freno',
    'Válvula de proporción de freno',
    'Depósito de líquido de freno',
    'Freno de mano (o freno de estacionamiento)',
    'Freno de tambor',
    'Discos ventilados',
    'Sensor de desgaste de pastillas',
    'Rotores de freno',
    'Pinza de freno',
    'Conjunto de frenado trasero',
    'Mordaza de freno',
    'Kit de resorte de tambor de freno',
    'Líquido de frenos',
    'Pistones de caliper',
    'Válvula de control de presión de freno',
    'Anillo reluctor de ABS',
    'Conector de freno de estacionamiento',
    'Pedal de freno',
    'Módulo de control de freno',
    'Tornillo de purga de freno',
    'Anillo de retención de tambor de freno',
]

In [33]:
productos = {
    f"SKU{i}": {
        '_id':str(ObjectId()),
        'descripcion':_mat[i],
        'tpu': random.randint(30,50),
        'personal_factor_cantidad':random.randint(2,10) / 100000,
        'personal_factor_tiempo':random.randint(1000,5000) / 100
    }
    
    for i in range(30)
}
cantidad_min, cantidad_max = 500, 5000
tiempo_min, tiempo_max = 6, 24
personal_base = 5



def generar_ot(dias_atras = 365, dias_adelante = 0, pers_req = True):
    
    producto = random.choice(list(productos.keys()))
    
    p = productos[producto]
    tpu = p['tpu']
    personal_factor_cantidad = p['personal_factor_cantidad']
    personal_factor_tiempo = p['personal_factor_tiempo']
    
    
    cantidad = random.randint(cantidad_min, cantidad_max)
    tiempo = random.uniform(tiempo_min, tiempo_max)

    personal_requerido = int(personal_base + personal_factor_cantidad * tpu * cantidad \
                             + personal_factor_tiempo / tiempo \
                             + random.uniform(-1, 1))
    
    # print(f"y = {personal_base} + {personal_factor_cantidad}*{tpu}({cantidad}) + {personal_factor_tiempo}/{tiempo:,.2f} + {random.randint(-2, 2)} = {personal_requerido:,}")

    personal_requerido = max(1, personal_requerido)

    inicio = datetime.datetime.now() - datetime.timedelta(days=random.randint(0, dias_atras)) + datetime.timedelta(days=random.randint(0, dias_adelante))
    final = inicio + datetime.timedelta(hours=tiempo)

    ot = {
        "producto": productos[producto]['_id'],
        "cantidad": cantidad,
        "inicio": inicio.strftime("%Y-%m-%d %H:%M"),
        "final": final.strftime("%Y-%m-%d %H:%M"),
        '_pseudo': producto,
    }

    
    return {**ot, "personalAsignado": personal_requerido} if pers_req else ot

def generar_datos(n=1000, dias_atras = 365, dias_adelante = 0, pers_req=True):
    ots = [generar_ot(dias_atras=dias_atras, dias_adelante=dias_adelante, pers_req=pers_req) for _ in tqdm(range(n))]
    return ots


data = pd.DataFrame(generar_datos(250_000))

100%|██████████| 250000/250000 [00:03<00:00, 74202.21it/s]


In [34]:
data['personalAsignado'].min(),data['personalAsignado'].max()

(5, 31)

# Evaluando los resultados de los datos creados

In [35]:
data['tiempo'] = (pd.to_datetime(data['final']) - pd.to_datetime(data['inicio'])).dt.total_seconds() / 3600

In [36]:
SKU = "SKU15"

In [37]:
import pandas as pd
import plotly.graph_objects as go

# Supongamos que tienes este DataFrame:
df = data[data['_pseudo']==SKU]

# Crear el gráfico 3D interactivo
fig = go.Figure(data=go.Scatter3d(
    x=df['cantidad'],
    y=df['tiempo'],
    z=df['personalAsignado'],
    mode='markers',
    marker=dict(
        size=1,
        color=df['personalAsignado'],  # Color según personal asignado
        colorscale='rdylbu',
        opacity=0.8
    )
))

# Personalizar el gráfico
fig.update_layout(
    scene = dict(
        xaxis_title='Cantidad',
        yaxis_title='Tiempo',
        zaxis_title='Personal Asignado'
    ),
    title=f'TPU = {productos[SKU]["tpu"]}'
)

# Mostrar el gráfico
fig.show()

# 1. Generando datos para la populacion de la base de datos

Para las acciones de populacion de bases de datos se mantendrá un avg de OT por dia de 5, para no sobrepoblar la bd.

Se generan datos para 5 años hacia atras.

In [38]:
materiales = [{'_id':p['_id'], 'tpu':p['tpu'], 'descripcion':p['descripcion']} for k,p in productos.items()]

In [39]:
colabs = [{'_id':str(ObjectId()),'nombre':f"{p['name']['first']} {p['name']['last']}",'foto':p['picture']['large']} for p in r.get('https://randomuser.me/api/?results=50').json()['results']]

In [40]:
ot = generar_datos(n = 5 * 365 * 5, dias_atras=365 * 5, pers_req=True)
mrp_ot = generar_datos(n = 5 * 60, dias_atras = 0, dias_adelante = 60, pers_req = False)

100%|██████████| 300/300 [00:00<00:00, 86754.77it/s]


Dado que la funcion unicamente devuelve el entero de la cantidad de personas asignadas, hay que convertirlo en una lista aleatoria de colaboradores en ot

In [41]:
for item in ot:
    num_ids = int(item['personalAsignado'])
    ids_unique = random.sample([d['_id'] for d in colabs], k=num_ids)
    item['personalAsignado'] = ids_unique

# 2. Entrenamiento de modelo

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [43]:
data['tpu'] = data['_pseudo'].apply(lambda x: productos.get(x).get('tpu'))

In [44]:
X = data[['tiempo','tpu','cantidad']]
y = data['personalAsignado']

### 2.1 Train test split

In [45]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

### 2.2.1 Modelo lineal

In [46]:
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [47]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5

print(f'Coeficiente R^2: {r2}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')

Coeficiente R^2: 0.6020736050314404
MAE: 2.174942883243709
MSE: 7.389687788675248
RMSE: 2.718398018810941


### 2.2.2 Modelo Random Forest

In [48]:
model2 = RandomForestRegressor()

model2.fit(X_train, y_train)

y_pred_rf = model2.predict(X_test)


r2 = r2_score(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)
mse = mean_squared_error(y_test, y_pred_rf)
rmse = mse**0.5

print(f'Coeficiente R^2: {r2}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')

Coeficiente R^2: 0.8337834638786694
MAE: 1.2529236376142376
MSE: 3.0867223757517834
RMSE: 1.7569070481251372


In [52]:
import joblib

joblib.dump(model2, "api\\services\\PA_Regressor\\random_forest.pkl")

['api\\services\\PA_Regressor\\random_forest.pkl']

# 3. Inserción de datos en MongoDB

In [24]:
from api.db import db

In [25]:
# db.ordenes.insert_many(ot)
# db.mrp.insert_many(mrp_ot)
# db.materiales.insert_many(materiales)
# db.plantilla.insert_many(colabs)

InsertManyResult(['671d9e8bd4a3e766a663e3bc', '671d9e8bd4a3e766a663e3bd', '671d9e8bd4a3e766a663e3be', '671d9e8bd4a3e766a663e3bf', '671d9e8bd4a3e766a663e3c0', '671d9e8bd4a3e766a663e3c1', '671d9e8bd4a3e766a663e3c2', '671d9e8bd4a3e766a663e3c3', '671d9e8bd4a3e766a663e3c4', '671d9e8bd4a3e766a663e3c5', '671d9e8bd4a3e766a663e3c6', '671d9e8bd4a3e766a663e3c7', '671d9e8bd4a3e766a663e3c8', '671d9e8bd4a3e766a663e3c9', '671d9e8bd4a3e766a663e3ca', '671d9e8bd4a3e766a663e3cb', '671d9e8bd4a3e766a663e3cc', '671d9e8bd4a3e766a663e3cd', '671d9e8bd4a3e766a663e3ce', '671d9e8bd4a3e766a663e3cf', '671d9e8bd4a3e766a663e3d0', '671d9e8bd4a3e766a663e3d1', '671d9e8bd4a3e766a663e3d2', '671d9e8bd4a3e766a663e3d3', '671d9e8bd4a3e766a663e3d4', '671d9e8bd4a3e766a663e3d5', '671d9e8bd4a3e766a663e3d6', '671d9e8bd4a3e766a663e3d7', '671d9e8bd4a3e766a663e3d8', '671d9e8bd4a3e766a663e3d9', '671d9e8bd4a3e766a663e3da', '671d9e8bd4a3e766a663e3db', '671d9e8bd4a3e766a663e3dc', '671d9e8bd4a3e766a663e3dd', '671d9e8bd4a3e766a663e3de', '6